In [4]:
import pandas as pd
import re

# === 參數區 ===
ptt_path = '../ptt_語料.csv'  # 檔案在上層資料夾
scam_path = '500精簡詐騙字詞_UTF8.csv'
output_path = '../ptt_語料_處理後.csv'

# === 詐騙關鍵字處理 ===
scam_keywords_df = pd.read_csv(scam_path)
scam_keywords = scam_keywords_df['詞語'].dropna().astype(str).str.strip().tolist()

# === 保險關鍵字 ===
insurance_keywords = ['產險', '投資型', '壽險', '保單']

def count_keywords(text, keywords):
    if pd.isna(text): return 0
    return sum(text.count(k) for k in keywords)

# === 載入資料 ===
ptt_df = pd.read_csv(ptt_path)
ptt_df['論壇'] = 'PTT'

# 標註是否主文
ptt_df['是否主文'] = ~ptt_df['來源'].str.contains('-回文')

# 發文者帳號（主文）
ptt_df['發文者帳號'] = ptt_df.apply(
    lambda row: row['內容'].split('作者')[1].split(' ')[0] if row['是否主文'] and '作者' in row['內容'] else None,
    axis=1
)

# 留言帳號（留言）
ptt_df['留言帳號'] = ptt_df.apply(
    lambda row: row['內容'].split(':')[0].replace('→', '').strip() if not row['是否主文'] and ':' in row['內容'] else None,
    axis=1
)

# 發文與留言時間擷取
def extract_time(row):
    if row['是否主文']:
        match = re.search(r'時間(.*?)\n', row['內容'])
        return match.group(1).strip() if match else None
    else:
        match = re.search(r'\d{2}/\d{2} \d{2}:\d{2}$', row['內容'])
        return match.group(0) if match else None

ptt_df['發文時間'] = ptt_df.apply(lambda r: extract_time(r) if r['是否主文'] else None, axis=1)
ptt_df['留言時間'] = ptt_df.apply(lambda r: extract_time(r) if not r['是否主文'] else None, axis=1)

# 發文與留言內容
ptt_df['發文內容'] = ptt_df.apply(lambda r: r['內容'] if r['是否主文'] else None, axis=1)
ptt_df['留言內容'] = ptt_df.apply(lambda r: r['內容'] if not r['是否主文'] else None, axis=1)

# 關鍵字統計
ptt_df['保險關鍵詞次數'] = ptt_df['內容'].apply(lambda x: count_keywords(x, insurance_keywords))
ptt_df['詐騙關鍵詞次數'] = ptt_df['內容'].apply(lambda x: count_keywords(x, scam_keywords))
ptt_df['是否提及特定公司'] = ptt_df['內容'].apply(lambda x: any(k in x for k in ['國泰', '大樹']))

# === 輸出結果 ===
ptt_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f'✅ 已完成清洗並輸出：{output_path}')


✅ 已完成清洗並輸出：../ptt_語料_處理後.csv
